In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import unary_union
import numpy as np
from sklearn.neighbors import BallTree
import math
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from matplotlib.patches import Patch
from shapely import wkt

In [2]:
FRP_FILTER = 20
RADIUS_FIRES = 200

In [3]:
with open('./token.txt', 'r') as f:
    TOKEN = f.read()
    px.set_mapbox_access_token(TOKEN)

In [4]:
df = pd.read_parquet('../data/fires_merged_comunas_timezone.parquet')

In [5]:
df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,comuna,acq_datetime_gmt_3
0,-23.820446,-70.320282,301.51,0.74,0.76,2013-01-01,448,N,VIIRS,n,1,285.54,2.38,N,2.0,ANTOFAGASTA,2013-01-01 01:48:00-03:00
1,-23.823833,-70.318871,306.90,0.74,0.76,2013-01-01,448,N,VIIRS,n,1,285.80,2.33,N,2.0,ANTOFAGASTA,2013-01-01 01:48:00-03:00
2,-26.430983,-69.475632,299.73,0.58,0.70,2013-01-01,448,N,VIIRS,n,1,279.61,2.86,N,2.0,DIEGO DE ALMAGRO,2013-01-01 01:48:00-03:00
3,-32.760929,-71.476440,309.70,0.52,0.67,2013-01-01,448,N,VIIRS,n,1,285.42,2.50,N,3.0,PUCHUNCAVI,2013-01-01 01:48:00-03:00
4,-34.624073,-71.000023,319.97,0.44,0.63,2013-01-01,448,N,VIIRS,n,1,290.28,2.27,N,0.0,CHIMBARONGO,2013-01-01 01:48:00-03:00


In [6]:
df = df[df['type'] == 0]

In [7]:
df['acq_datetime_gmt_3'] = pd.to_datetime(df['acq_datetime_gmt_3'])

In [8]:
df['acq_datetime_gmt_3'].min(), df['acq_datetime_gmt_3'].max()

(Timestamp('2013-01-01 01:48:00-0300', tz='America/Santiago'),
 Timestamp('2023-01-31 16:38:00-0300', tz='America/Santiago'))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287968 entries, 4 to 465149
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype                           
---  ------              --------------   -----                           
 0   latitude            287968 non-null  float64                         
 1   longitude           287968 non-null  float64                         
 2   brightness          287968 non-null  float64                         
 3   scan                287968 non-null  float64                         
 4   track               287968 non-null  float64                         
 5   acq_date            287968 non-null  datetime64[us]                  
 6   acq_time            287968 non-null  int64                           
 7   satellite           287968 non-null  object                          
 8   instrument          287968 non-null  object                          
 9   confidence          287968 non-null  object                     

In [10]:
"""plt.figure(figsize=(10, 6))
sns.histplot(data=df, x="frp", binrange=(df["frp"].quantile(0.01), df["frp"].quantile(0.99)), bins=100)
plt.title('Distribution of FRP Values')
plt.xlabel('FRP')
plt.ylabel('Count')
plt.tight_layout()
plt.show();"""

'plt.figure(figsize=(10, 6))\nsns.histplot(data=df, x="frp", binrange=(df["frp"].quantile(0.01), df["frp"].quantile(0.99)), bins=100)\nplt.title(\'Distribution of FRP Values\')\nplt.xlabel(\'FRP\')\nplt.ylabel(\'Count\')\nplt.tight_layout()\nplt.show();'

In [11]:
a = df[df['frp'] > FRP_FILTER].copy()

In [12]:
a['year'] = a['acq_datetime_gmt_3'].dt.year

In [13]:
a.shape

(78725, 18)

In [14]:
fig = px.scatter_mapbox(a[(a['type'] == 0) & ((a['comuna'] == 'CONCEPCION') | (a['comuna'] == 'VICTORIA') | (a['comuna'] == 'FLORIDA')  | (a['comuna'] == 'TOME'))], lat="latitude", lon="longitude", zoom=3, color='frp', title='Wildfires')
fig.update_layout(mapbox_style='satellite', title='Clusters Analysis for Static land source', height=800)
fig.show();

/tmp/ipykernel_150691/1848473221.py:1: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(a[(a['type'] == 0) & ((a['comuna'] == 'CONCEPCION') | (a['comuna'] == 'VICTORIA') | (a['comuna'] == 'FLORIDA')  | (a['comuna'] == 'TOME'))], lat="latitude", lon="longitude", zoom=3, color='frp', title='Wildfires')


In [15]:
x = a.copy()
x.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,comuna,acq_datetime_gmt_3,year
31,-33.305400,-70.752300,326.70,2.20,1.40,2013-01-01,1521,Terra,MODIS,n,6.03,308.00,31.90,D,0.0,LAMPA,2013-01-01 12:21:00-03:00,2013
43,-33.309000,-70.753200,324.80,3.10,1.70,2013-01-01,1800,Aqua,MODIS,n,6.03,310.40,37.70,D,0.0,LAMPA,2013-01-01 15:00:00-03:00,2013
44,-34.471400,-71.336800,318.20,3.50,1.80,2013-01-01,1800,Aqua,MODIS,n,6.03,306.10,43.20,D,0.0,PALMILLA,2013-01-01 15:00:00-03:00,2013
48,-37.857079,-71.161934,333.99,0.48,0.40,2013-01-01,1848,N,VIIRS,n,1,301.94,20.16,D,0.0,SANTA BARBARA,2013-01-01 15:48:00-03:00,2013
54,-33.296406,-70.749161,355.82,0.40,0.44,2013-01-01,1854,N,VIIRS,n,1,315.79,23.86,D,0.0,LAMPA,2013-01-01 15:54:00-03:00,2013


In [16]:
x['mes'] = x['acq_date'].dt.month

In [17]:
x = x[(x['mes'] >= 11) | (x['mes'] <= 3)]

In [18]:
x.shape

(59722, 19)

In [19]:
fig = px.scatter_mapbox(x[(x['type'] == 0) & ((x['comuna'] == 'VICTORIA') )], lat="latitude", lon="longitude", zoom=3, color='frp', title='Wildfires')
fig.update_layout(mapbox_style='satellite', title='Clusters Analysis for Static land source', height=800)
fig.show();

/tmp/ipykernel_150691/2421778495.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [20]:

def find_intersections_across_years(df, radius_meters=300):
    """
    Find points that intersect with points from other years, using a 300m radius.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Contains latitude, longitude, and year columns
    radius_meters : float
        Radius in meters to consider points as intersecting
    
    Returns:
    --------
    pd.DataFrame : All points that intersect with points from other years
    """
    # Earth radius in meters
    EARTH_RADIUS = 6371000
    
    # Convert radius to radians for haversine distance
    radius_radians = radius_meters / EARTH_RADIUS
    
    # Group by year
    years = sorted(df['year'].unique())
    
    # Keep track of intersecting points
    intersecting_indices = set()
    
    # Process each year
    for year in tqdm(years, desc="Processing years"):
        year_points = df[df['year'] == year]
        
        # Skip if empty
        if len(year_points) == 0:
            continue
            
        # Convert lat/lon to radians for the BallTree
        year_coords = np.radians(year_points[['latitude', 'longitude']].values)
        
        # Create BallTree for current year
        tree = BallTree(year_coords, metric='haversine')
        
        # Check against all other years
        for other_year in years:
            if other_year == year:
                continue
                
            other_points = df[df['year'] == other_year]
            
            # Skip if empty
            if len(other_points) == 0:
                continue
                
            # Convert other year points to radians
            other_coords = np.radians(other_points[['latitude', 'longitude']].values)
            
            # Find neighbors within radius
            indices = tree.query_radius(other_coords, radius_radians)
            
            # Add points from current year that have neighbors
            for i, idx_array in enumerate(indices):
                if len(idx_array) > 0:
                    # Get the original dataframe indices
                    for idx in idx_array:
                        intersecting_indices.add(year_points.iloc[idx].name)
    
    # Return the intersecting points
    return df.loc[list(intersecting_indices)]

In [21]:
a.shape

(78725, 18)

In [22]:
b = find_intersections_across_years(a, RADIUS_FIRES)

Processing years: 100%|██████████| 11/11 [00:06<00:00,  1.63it/s]


In [23]:
b.shape

(10443, 18)

In [24]:
"""fig = px.scatter_mapbox(b[(b['type'] == 0) & (b['comuna'] == 'PENCO')], lat="latitude", lon="longitude", zoom=3, color='frp', title='Wildfires')
fig.update_layout(mapbox_style='satellite', title='Clusters Analysis for Static land source', height=800)
fig.show();"""

'fig = px.scatter_mapbox(b[(b[\'type\'] == 0) & (b[\'comuna\'] == \'PENCO\')], lat="latitude", lon="longitude", zoom=3, color=\'frp\', title=\'Wildfires\')\nfig.update_layout(mapbox_style=\'satellite\', title=\'Clusters Analysis for Static land source\', height=800)\nfig.show();'

In [25]:
def getClusterData(df):
    def parse_time(value):
        hours = value // 100
        minutes = value % 100
        return pd.to_timedelta(f"{hours} hours {minutes} minutes")

    clustered_data = pd.DataFrame()
    mbr_data = pd.DataFrame()

    min_samples = 5
    epsilon = 0.010

    subset = df[['latitude', 'longitude', 'comuna', 'frp']].copy()
    
    if len(subset) >= min_samples:
        db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(subset[['latitude', 'longitude']])
        subset['cluster'] = db.labels_

        # Get the MBR data
        for cluster_label in np.unique(db.labels_):
            if cluster_label == -1:
                continue
            cluster_points = subset[subset['cluster'] == cluster_label][['latitude', 'longitude']]
            comuna_points = subset[subset['cluster'] == cluster_label][['comuna', 'frp']]

            mbr = MultiPoint(cluster_points.values).envelope

            comuna_labels = comuna_points['comuna'].unique()
            max_frp = comuna_points['frp'].max()
            min_frp = comuna_points['frp'].min()
            mean_frp = comuna_points['frp'].mean()

            mbr_df = pd.DataFrame({
                'cluster': [cluster_label],
                'mbr': [mbr],
                'comunas': [comuna_labels],
                'max_frp': max_frp,
                'min_frp': min_frp,
                'mean_frp': mean_frp
            })
            mbr_data = pd.concat([mbr_data, mbr_df], axis=0)
        clustered_data = pd.concat([clustered_data, subset], ignore_index=True)

    return (clustered_data.reset_index(drop=True), mbr_data.reset_index(drop=True))

In [26]:
clusters, mbrs = getClusterData(b)

In [27]:
mbrs.head()

,cluster,mbr,comunas,max_frp,min_frp,mean_frp
0,0,"POLYGON ((-38.8237 -72.3797, -38.814835 -72.37...",[VILCUN],98.60,35.25,65.500000
1,1,"POLYGON ((-38.4655 -72.8389, -38.430832 -72.83...",[GALVARINO],958.40,20.30,86.992745
2,2,"POLYGON ((-26.3829 -70.093, -26.3691 -70.093, ...",[CHANARAL],100.80,20.19,32.328929
3,3,"POLYGON ((-36.9935 -71.9645, -36.983803 -71.96...",[EL CARMEN],297.20,20.04,78.990541
4,4,"POLYGON ((-36.7806 -71.952568, -36.7597 -71.95...",[PINTO],191.37,21.04,51.371471


In [28]:
geo_df = gpd.GeoDataFrame(geometry=mbrs['mbr'], data=mbrs.drop(columns=['mbr']))

In [29]:
geo_df.head()

,cluster,comunas,max_frp,min_frp,mean_frp,geometry
0,0,[VILCUN],98.60,35.25,65.500000,"POLYGON ((-38.8237 -72.3797, -38.81484 -72.379..."
1,1,[GALVARINO],958.40,20.30,86.992745,"POLYGON ((-38.4655 -72.8389, -38.43083 -72.838..."
2,2,[CHANARAL],100.80,20.19,32.328929,"POLYGON ((-26.3829 -70.093, -26.3691 -70.093, ..."
3,3,[EL CARMEN],297.20,20.04,78.990541,"POLYGON ((-36.9935 -71.9645, -36.9838 -71.9645..."
4,4,[PINTO],191.37,21.04,51.371471,"POLYGON ((-36.7806 -71.95257, -36.7597 -71.952..."


In [30]:
geo_df.to_parquet('./incendios_por_comuna_frp.parquet', index=None)

In [31]:
geo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   cluster   491 non-null    int64   
 1   comunas   491 non-null    object  
 2   max_frp   491 non-null    float64 
 3   min_frp   491 non-null    float64 
 4   mean_frp  491 non-null    float64 
 5   geometry  491 non-null    geometry
dtypes: float64(3), geometry(1), int64(1), object(1)
memory usage: 23.1+ KB


In [32]:
clusters.head()

,latitude,longitude,comuna,frp,cluster
0,-35.516949,-72.10276,SAN JAVIER,27.21,-1
1,-34.296900,-71.38000,PICHIDEGUA,29.50,-1
2,-38.816200,-72.37650,VILCUN,98.60,0
3,-38.817400,-72.37970,VILCUN,95.10,0
4,-38.441700,-72.79540,GALVARINO,68.20,1


In [33]:
clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10443 entries, 0 to 10442
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   10443 non-null  float64
 1   longitude  10443 non-null  float64
 2   comuna     10443 non-null  object 
 3   frp        10443 non-null  float64
 4   cluster    10443 non-null  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 408.1+ KB


In [34]:
clusters_sample = clusters.sample(1000)

In [35]:
fig = px.scatter_mapbox(
    clusters_sample,
    lat="latitude",
    lon="longitude",
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=10,
    title='Fires Clustered by DBSCAN'
)

for i, row in geo_df.iterrows():
    min_lat, min_lon, max_lat, max_lon = row['geometry'].bounds

    rectangle = go.Scattermapbox(
        lat=[min_lat, max_lat, max_lat, min_lat, min_lat],
        lon=[min_lon, min_lon, max_lon, max_lon, min_lon],
        mode="lines",
        line=dict(color='white'),
        fill='toself',
        fillcolor='rgba(255,0,0,0.3)',
        showlegend=False
    )
    fig.add_trace(rectangle)

fig.update_layout(
    mapbox_style='satellite',
    title='Fires Clustered by DBSCAN',
    height=800
)

# Show the plot
fig.show()

/tmp/ipykernel_150691/3620456498.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/tmp/ipykernel_150691/3620456498.py:14: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

